## Import packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.model_selection import train_test_split
import random
import shutil

## Random Images Selection Functions
- get_random_images() => Get random images from a folder of images
- store_images() => Store images using image paths in folder 'test_images'

In [2]:
def get_random_images(image_folder, num_images):
    image_paths = []
    while (len(image_paths) < num_images):
        for dir1 in os.listdir(image_folder):
            for dir2 in os.listdir(os.path.join(image_folder, dir1)):
                image_path = os.path.join(image_folder, dir1, dir2)
                if (random.randint(0,10000) > 9999 and len(image_paths) < 10):
                    image_paths.append(image_path)

    return image_paths

In [3]:
def store_images(image_paths):
    os.makedirs('test_images')
    for i in range(0, len(image_paths)):
        shutil.copy(image_paths[i], 'test_images')

### Select random images from train_images and store in test_images
Make sure 'test_images' directory does not exist when running this

In [4]:
num_of_test_images = 10     # Change this to the number of images you want to test

test_images = get_random_images('train_images', num_of_test_images)
store_images(test_images)
print(test_images)

['train_images/video_2/10321.jpg', 'train_images/video_1/6593.jpg', 'train_images/video_0/4450.jpg', 'train_images/video_0/12277.jpg', 'train_images/video_1/505.jpg', 'train_images/video_1/237.jpg', 'train_images/video_0/6211.jpg', 'train_images/video_2/8577.jpg', 'train_images/video_2/9842.jpg', 'train_images/video_1/9497.jpg']


## Login to Weights & Biases AI and download model weights to detect with

In [5]:
import wandb

try:
    api_key = "f72dca236cb3497e771bf6c5ea119c25fcfb1b10"
    wandb.login(key=api_key)
    anonymous = None
    print('Logged in!')
except:
    wandb.login(anonymous='must')
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: justjo3l. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/joeljose/.netrc


Logged in!


### Install weights in artifacts directory
NOTE: Remove the ':v0' in the downloaded model directory name

In [6]:
run = wandb.init()
artifact = run.use_artifact('justjo3l/YOLOR/run_1w24g32k_model:v0', type='model')
artifact_dir = artifact.download()
print(artifact_dir)

wandb: Downloading large artifact run_1w24g32k_model:v0, 315.54MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.2


./artifacts/run_1w24g32k_model:v0


## Clone YOLOv7 git repo and install requirements
NOTE: Replace the detect.py in yolov7 with the detect.py in the Crown-Of-Throrns repository to get COTS count and differently colored bounding boxes for individual COTS

In [7]:
!rm -r yolov7
!git clone https://github.com/WongKinYiu/yolov7.git # clone
%cd yolov7
%pip install -qr requirements.txt  # install

rm: yolov7: No such file or directory
Cloning into 'yolov7'...
remote: Enumerating objects: 998, done.
remote: Total 998 (delta 0), reused 0 (delta 0), pack-reused 998
Receiving objects: 100% (998/998), 69.77 MiB | 16.88 MiB/s, done.
Resolving deltas: 100% (466/466), done.
/Users/joeljose/Desktop/Coding Projects/python/comp9517/project/Crown-of-Throrns/yolov7
Note: you may need to restart the kernel to use updated packages.


#### Change directory to yolov7/

In [8]:
%cd yolov7

[Errno 2] No such file or directory: 'yolov7'
/Users/joeljose/Desktop/Coding Projects/python/comp9517/project/Crown-of-Throrns/yolov7


## Run detection on images in test_images
Results will be stored in the yolov7/runs/detect/exp folders

In [10]:
!python detect.py --weights ../artifacts/run_1w24g32k_model/best.pt --conf 0.1 --source ../test_images

Namespace(weights=['../artifacts/run_1w24g32k_model/best.pt'], source='../test_images', img_size=640, conf_thres=0.1, iou_thres=0.45, device='', view_img=False, save_txt=False, save_conf=False, nosave=False, classes=None, agnostic_nms=False, augment=False, update=False, project='runs/detect', name='exp', exist_ok=False, no_trace=False)
YOLOR 🚀 2022-11-17 torch 1.13.0 CPU

Fusing layers... 
IAuxDetect.fuse
[W NNPACK.cpp:53] Could not initialize NNPACK! Reason: Unsupported hardware.
/Users/joeljose/opt/anaconda3/envs/cotsenv/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3191.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 819 layers, 164816216 parameters, 0 gradients, 225.4 GFLOPS
 Convert model to Traced-model... 
 traced_script_module